## Paper visualizations

In [ ]:
!pip install --user neural_renderer_pytorch

In [44]:
import os
import imageio
import trimesh
import torch
import numpy as np

import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
%matplotlib inline

import neural_renderer as nr
from scipy.spatial import cKDTree as KDTree
from datasets import make_data_instance_from_stl

from models import *

import pdb

def get_rotate_matrix(rotation_angle1):
    cosval = np.cos(rotation_angle1)
    sinval = np.sin(rotation_angle1)

    rotation_matrix_x = np.array([[1, 0, 0, 0],
                                  [0, cosval, -sinval, 0],
                                  [0, sinval, cosval, 0],
                                  [0, 0, 0, 1]])
    rotation_matrix_y = np.array([[cosval, 0, sinval, 0],
                                  [0, 1, 0, 0],
                                  [-sinval, 0, cosval, 0],
                                  [0, 0, 0, 1]])
    rotation_matrix_z = np.array([[cosval, -sinval, 0, 0],
                                  [sinval, cosval, 0, 0],
                                  [0, 0, 1, 0],
                                  [0, 0, 0, 1]])
    scale_y_neg = np.array([
        [1, 0, 0, 0],
        [0, -1, 0, 0],
        [0, 0, 1, 0],
        [0, 0, 0, 1]
    ])

    neg = np.array([
        [-1, 0, 0, 0],
        [0, -1, 0, 0],
        [0, 0, -1, 0],
        [0, 0, 0, 1]
    ])
    # y,z swap = x rotate -90, scale y -1
    # new_pts0[:, 1] = new_pts[:, 2]
    # new_pts0[:, 2] = new_pts[:, 1]
    #
    # x y swap + negative = z rotate -90, scale y -1
    # new_pts0[:, 0] = - new_pts0[:, 1] = - new_pts[:, 2]
    # new_pts0[:, 1] = - new_pts[:, 0]

    # return np.linalg.multi_dot([rotation_matrix_z, rotation_matrix_y, rotation_matrix_y, scale_y_neg, rotation_matrix_z, scale_y_neg, rotation_matrix_x])
    return np.linalg.multi_dot([neg, rotation_matrix_z, rotation_matrix_z, scale_y_neg, rotation_matrix_x])

def get_projection_matricies(az, el, distance_ratio, roll = 0, focal_length=35, img_w=137, img_h=137):
    """
    Calculate 4x3 3D to 2D projection matrix given viewpoint parameters.
    Code from "https://github.com/Xharlie/DISN"
    """

    F_MM = focal_length  # Focal length
    SENSOR_SIZE_MM = 32.
    PIXEL_ASPECT_RATIO = 1.  # pixel_aspect_x / pixel_aspect_y
    RESOLUTION_PCT = 100.
    SKEW = 0.
    CAM_MAX_DIST = 1.75
    CAM_ROT = np.asarray([[1.910685676922942e-15, 4.371138828673793e-08, 1.0],
                      [1.0, -4.371138828673793e-08, -0.0],
                      [4.371138828673793e-08, 1.0, -4.371138828673793e-08]])

    # Calculate intrinsic matrix.
    scale = RESOLUTION_PCT / 100
    # print('scale', scale)
    f_u = F_MM * img_w * scale / SENSOR_SIZE_MM
    f_v = F_MM * img_h * scale * PIXEL_ASPECT_RATIO / SENSOR_SIZE_MM
    # print('f_u', f_u, 'f_v', f_v)
    u_0 = img_w * scale / 2
    v_0 = img_h * scale / 2
    K = np.matrix(((f_u, SKEW, u_0), (0, f_v, v_0), (0, 0, 1)))

    # Calculate rotation and translation matrices.
    # Step 1: World coordinate to object coordinate.
    sa = np.sin(np.radians(-az))
    ca = np.cos(np.radians(-az))
    se = np.sin(np.radians(-el))
    ce = np.cos(np.radians(-el))
    R_world2obj = np.transpose(np.matrix(((ca * ce, -sa, ca * se),
                                          (sa * ce, ca, sa * se),
                                          (-se, 0, ce))))

    # Step 2: Object coordinate to camera coordinate.
    R_obj2cam = np.transpose(np.matrix(CAM_ROT))
    R_world2cam = R_obj2cam * R_world2obj
    cam_location = np.transpose(np.matrix((distance_ratio * CAM_MAX_DIST,
                                           0,
                                           0)))
    T_world2cam = -1 * R_obj2cam * cam_location

    # Step 3: Fix blender camera's y and z axis direction.
    R_camfix = np.matrix(((1, 0, 0), (0, -1, 0), (0, 0, -1)))
    R_world2cam = R_camfix * R_world2cam
    T_world2cam = R_camfix * T_world2cam

    RT = np.hstack((R_world2cam, T_world2cam))
    # finally, consider roll
    cr = np.cos(np.radians(roll))
    sr = np.sin(np.radians(roll))
    R_z = np.matrix(((cr, -sr, 0),
                  (sr, cr, 0),
                  (0, 0, 1)))

    rot_mat = get_rotate_matrix(-np.pi / 2)

    return K, R_z@RT@rot_mat

def load_fld(fld_path):
    '''
     Takes a path to generated fld file with following colomns: x,y,z,p,k,omega,nut
                and converts it into a geometric data instance.
    '''

    fld = np.genfromtxt(fld_path, delimiter=',', skip_header=1)
    np.random.shuffle(fld)
    fld[fld > 10e5] = np.nan
    fld = fld[~np.isnan(fld).any(axis=1)]
    answers = fld[:, 3:]

    """
    mean_values = [-2.06707869e+00, 1.04133005e-01, 2.17513919e+02, 6.04485806e-05]
    std_values = [3.71674873e+00, 4.93675056e-02, 1.10871494e+02, 2.63155496e-05]
    for f in range(answers.shape[1]):
        answers[:, f] = (answers[:, f] - mean_values[f]) / std_values[f]
    """

    stl_path = fld_path.replace('fld', 'stl', 1)[:-9] + '.stl'
    mesh = trimesh.load(stl_path)
    # reinterpolate features on mesh
    fld_tree = KDTree(fld[:, :3])
    distances, indeces = fld_tree.query(mesh.vertices, k=1)
    interpolations = answers[indeces].squeeze()

    return mesh, interpolations

def load_predicted(ply_path):
    '''
     Takes a path to generated fld file with following colomns: x,y,z,p,k,omega,nut
                and converts it into a geometric data instance.
    '''

    answers_path = ply_path.replace('meshes', 'predictions', 1)[:-4] + '.npy'
    answers = np.load(answers_path)
    mesh = trimesh.load(ply_path)

    return mesh, answers


def interpolate_on_faces(field, faces):
    #TODO: no batch support for now
    nv = field.shape[0]
    nf = faces.shape[0]
    field = field.reshape((nv, 1))
    # pytorch only supports long and byte tensors for indexing
    face_coordinates = field[faces.long()].squeeze(0)
    centroids = 1.0/3 * torch.sum(face_coordinates, 1)
    return centroids.squeeze(-1)

def visualize(vertices, faces, fields, field_to_visualize = 0, 
              img_resolution = 1200, azimuth = 210, elevation=10, distance_ratio = 0.8, colormap=cm.jet, 
              color_blind=False):
    """
    Interface to neural_render to produce nice visualizations. It requires GPU.

    Inputs:
    vertices in [V,3]
    faces in [F,3]
    fields in [V,3]
    (ideally you can substitute this with a torch_geometric.data.Data object. 
    I didn't because I don't have it installed)

    Output:
    Image in [img_resolution, img_resolution, 3]
    """

    # first set up camera
    intrinsic, extrinsic =  get_projection_matricies(azimuth, elevation, distance_ratio, img_w=img_resolution, img_h=img_resolution)

    K_cuda = torch.tensor(intrinsic[np.newaxis, :, :].copy()).float().cuda().unsqueeze(0)
    R_cuda = torch.tensor(extrinsic[np.newaxis, 0:3, 0:3].copy()).float().cuda().unsqueeze(0)
    t_cuda = torch.tensor(extrinsic[np.newaxis, np.newaxis, 0:3, 3].copy()).float().cuda().unsqueeze(0)

    # initialize renderer
    renderer = nr.Renderer(image_size = img_resolution, orig_size = img_resolution, K=K_cuda, R=R_cuda, t=t_cuda, anti_aliasing=True)

    # now move vertices, faces to GPU
    verts_dr = torch.tensor(vertices.copy(), dtype=torch.float32, requires_grad = False).cuda()
    faces_dr = torch.tensor(faces.copy()).cuda()
    field_dr = torch.tensor(fields[:, field_to_visualize].copy(),dtype=torch.float32, requires_grad = False).cuda()
    # interpolate field on traingle center
    field_on_faces = interpolate_on_faces(field_dr, faces_dr)

    #TODO: find good values here? Maybe across the dataset to make visualization consistent? or this is good enough? I am not sure...
    norm = mpl.colors.Normalize(vmin= -6, vmax=6)
    cmap = colormap
    m = cm.ScalarMappable(norm=norm, cmap=cmap)
    # field_on_faces = torch.clamp((field_on_faces-field_min)/(field_max-field_min),0,1)

    textures_dr = torch.ones(faces_dr.shape[0], 1, 1, 1, 3, dtype=torch.float32).cuda()
    # feel free to pick your favorite color map here, I used this one for Sanity check, maybe we can  use another one here??
    
    if not color_blind:
        textures_dr[:,0,0,0, :] = torch.tensor(list(map(m.to_rgba,  field_on_faces.cpu().detach())), dtype=torch.float32).cuda()[:, :3]

    images_out, depth, alpha = renderer(verts_dr.unsqueeze(0), faces_dr.unsqueeze(0), textures_dr.unsqueeze(0))
    images_out = torch.cat([images_out[0], alpha])
    image_out_export = 255*images_out.detach().cpu().numpy().transpose((1, 2, 0))
    return image_out_export

In [42]:
def make_data_instance_from_ply(path):
    mesh = trimesh.load(path)
    edge_attr = [mesh.vertices[a] - mesh.vertices[b] for a, b in mesh.edges]
    data = torch_geometric.data.Data(x  = torch.tensor(mesh.vertices, dtype=torch.float), 
                                     pos= torch.tensor(mesh.vertices, dtype=torch.float),
                                     face = torch.tensor(mesh.faces, dtype=torch.long).t(),
                                     edge_attr = torch.tensor(edge_attr, dtype=torch.float),
                                     edge_index= torch.tensor(mesh.edges, dtype=torch.long).t().contiguous())
    return data


def process_mesh(path, suffix="", model=None, out_dir=None, take_from_fld=True, prefields=None,
                                  norm_field=False,  **kwargs):
    FLD_PATH = path
    if take_from_fld:
        mesh, fields = load_fld(FLD_PATH)
    else:
        mesh, fields = load_predicted(FLD_PATH)
    
    if out_dir is None:
        out_dir = os.path.join(*FLD_PATH.split("/")[:-2], 'output')
        
    if model is not None:
        if suffix == "":
            suffix = '_predicted'
        if take_from_fld:
            data_instance = make_data_instance_from_stl(path)
        else:
            data_instance = make_data_instance_from_ply(path)
        fields = model(data_instance.to('cuda:0')).cpu().detach().numpy()
        
    if prefields is not None:
        fields = prefields

    if norm_field:
        fields = (fields - np.mean(fields[:, 0])) / np.std(fields[:, 0])
        
    image = visualize(mesh.vertices, mesh.faces, fields, **kwargs)

    image_filename = os.path.join(out_dir, FLD_PATH.split("/")[-1][:-4]) + suffix + ".png" 
    imageio.imwrite(image_filename, image.astype(np.uint8))
    
def process_dir(path):
    files = os.listdir(path)
    for name in files:
        process_mesh(os.path.join(path, name))

In [43]:
model = SplineCNN8Residuals(3)
model.load_state_dict(torch.load("Expirements/Networks15/normilized_full_latest.nn"))
model.to('cuda:0')
model = model.eval()
print("done")

done


# Optimization Plots

In [37]:
def visualize_mesh_opt(path, out, take_each=3, baseline=False, **kwargs):
    if not os.path.exists(out):
        os.makedirs(out)
    
    path = os.path.join(path, 'meshes')
    files = [os.path.join(path, name) for name in filter(lambda x: x[0] == '0' and x[-3:] == "ply", os.listdir(root))]
    for idx in range(0, len(files), take_each): #(list(range(0, 30, take_each)) +
        inp_path = files[idx]
        data_instance = make_data_instance_from_ply(inp_path)
        
        if baseline:
            if idx == 0:
                edge_attrs = data_instance.edge_attr
                continue
            else:
                data_instance.edge_attr = edge_attrs

        fields = model(data_instance.to('cuda:0')).cpu().detach().numpy()
        
        process_mesh(inp_path, suffix='_intr', prefields=fields,
                     out_dir=out, norm_field=True, **kwargs,
                     azimuth=240, elevation=5, take_from_fld=False)
        process_mesh(inp_path, suffix='_angl', prefields=fields,
                     out_dir=out, norm_field=True, **kwargs,
                     take_from_fld=False)
        process_mesh(inp_path, suffix='_pery', prefields=fields,
                     out_dir=out, norm_field=True, **kwargs,
                     azimuth=-270, elevation=90, take_from_fld=False)
        process_mesh(inp_path, suffix='_perz', prefields=fields,
                     out_dir=out, norm_field=True, **kwargs,
                     azimuth=-270, elevation=0, take_from_fld=False)
        process_mesh(inp_path, suffix='_perx', prefields=fields,
                     out_dir=out, norm_field=True, **kwargs,
                     azimuth=180, elevation=0, take_from_fld=False)
#         process_mesh(inp_path, suffix='_spoiler2', prefields=fields,
#                      out_dir=out, norm_field=True,
#                      azimuth=-45, elevation=0, take_from_fld=False)

In [42]:
for idx in [535]:
    visualize_mesh_opt('Expirements/OptimizationPaper/AfterMeeting/FreeformDrag/%dminus/' % idx,
                       'Expirements/Visualizations/Paper/OptimizationDifferent/%dFreeFormMinus/' % idx, 
                        baseline=True, take_each=3)

In [ ]:
for idx in [69]:
    visualize_mesh_opt('Expirements/OptimizationPaper/AfterMeeting/UmetamiDrag2/%04d/' % idx,
                       'Expirements/Visualizations/Paper/OptimizationDifferent/%dUmetami/' % idx, 
                        baseline=True, take_each=3)

In [ ]:
for idx in [535, 69, 32, 162, 61]:
    visualize_mesh_opt('Expirements/OptimizationPaper/AfterMeeting/UmetamiDrag2/%04d/' % idx,
                       'Expirements/Visualizations/Paper/ForVideos/Umetami%d/' % idx, take_each=1, baseline=True)

In [52]:
for idx in [175]:
    visualize_mesh_opt('Expirements/OptimizationPaper/AfterMeeting/FreeformDrag/%03d/' % idx,
                       'Expirements/Visualizations/Paper/HighRes/FreeForm%04d/' % idx, take_each=19)

In [60]:
for idx in [175]:
    visualize_mesh_opt('Expirements/OptimizationPaper/AfterMeeting/DeepSDFDrag//%03d/' % idx,
                       'Expirements/Visualizations/Paper/HighRes/DeepSDF%04d/' % idx, take_each=100)

In [ ]:
fields = np.load(a.replace('meshes', 'predictions').replace('ply', 'npy'))

In [ ]:
fields

In [10]:
plya

'Expirements/OptimizationPaper/AfterMeeting/FreeformDrag/175/meshes/00003.ply'

In [ ]:
visualize_mesh_opt('Expirements/OptimizationPaper/AfterMeeting/DeepSDFDrag/175/',
                   'Expirements/Visualizations/Paper/OptimizationDifferent/175SpoilerDisappear')

# Hotmap Visualizations

In [ ]:
out_dir = "Expirements/Visualizations/Paper/PredictionComparison/afmhotNormFull_1"
colormap = cm.afmhot

process_mesh('/cvlabdata2/home/artem/Data/cars_remeshed_dsdf/outputs/fld/0001_0015.fld', 
             out_dir=out_dir, norm_field=True, colormap=colormap)
process_mesh('/cvlabdata2/home/artem/Data/cars_remeshed_dsdf/outputs/fld/0001_0015.fld', 
             out_dir=out_dir, norm_field=True, model=model, colormap=colormap)

process_mesh('/cvlabdata2/home/artem/Data/cars_remeshed_dsdf/outputs/fld/0002_0015.fld', 
             out_dir=out_dir, norm_field=True, colormap=cm.afmhot)
process_mesh('/cvlabdata2/home/artem/Data/cars_remeshed_dsdf/outputs/fld/0002_0015.fld', 
             out_dir=out_dir, norm_field=True, model=model, colormap=colormap)

process_mesh('/cvlabdata2/home/artem/Data/cars_remeshed_dsdf/outputs/fld/0003_0015.fld', 
             out_dir=out_dir, norm_field=True)
process_mesh('/cvlabdata2/home/artem/Data/cars_remeshed_dsdf/outputs/fld/0003_0015.fld', 
             out_dir=out_dir, norm_field=True, model=model, colormap=colormap)

process_mesh('/cvlabdata2/home/artem/Data/cars_remeshed_dsdf/outputs/fld/0004_0015.fld', 
             out_dir=out_dir, norm_field=True, colormap=colormap)
process_mesh('/cvlabdata2/home/artem/Data/cars_remeshed_dsdf/outputs/fld/0004_0015.fld', 
             out_dir=out_dir, norm_field=True, model=model, colormap=colormap)

In [ ]:
out_dir = "Expirements/Visualizations/Paper/PredictionComparison/hotNormFull_1"
colormap = cm.hot

process_mesh('/cvlabdata2/home/artem/Data/cars_remeshed_dsdf/outputs/fld/0001_0015.fld', 
             out_dir=out_dir, norm_field=True, colormap=colormap)
process_mesh('/cvlabdata2/home/artem/Data/cars_remeshed_dsdf/outputs/fld/0001_0015.fld', 
             out_dir=out_dir, norm_field=True, model=model, colormap=colormap)

process_mesh('/cvlabdata2/home/artem/Data/cars_remeshed_dsdf/outputs/fld/0002_0015.fld', 
             out_dir=out_dir, norm_field=True, colormap=cm.afmhot)
process_mesh('/cvlabdata2/home/artem/Data/cars_remeshed_dsdf/outputs/fld/0002_0015.fld', 
             out_dir=out_dir, norm_field=True, model=model, colormap=colormap)

process_mesh('/cvlabdata2/home/artem/Data/cars_remeshed_dsdf/outputs/fld/0003_0015.fld', 
             out_dir=out_dir, norm_field=True)
process_mesh('/cvlabdata2/home/artem/Data/cars_remeshed_dsdf/outputs/fld/0003_0015.fld', 
             out_dir=out_dir, norm_field=True, model=model, colormap=colormap)

process_mesh('/cvlabdata2/home/artem/Data/cars_remeshed_dsdf/outputs/fld/0004_0015.fld', 
             out_dir=out_dir, norm_field=True, colormap=colormap)
process_mesh('/cvlabdata2/home/artem/Data/cars_remeshed_dsdf/outputs/fld/0004_0015.fld', 
             out_dir=out_dir, norm_field=True, model=model, colormap=colormap)

In [39]:
out_dir = "Expirements/Visualizations/Paper/HighRes"
colormap = cm.jet

for idx in [411]:
    inp_path = '/cvlabdata2/home/artem/Data/cars_remeshed_dsdf/outputs/fld/%04d_0015.fld' % idx
    if os.path.exists(inp_path):
        process_mesh(inp_path, suffix='_spoilerHR_-120_10',
                     out_dir=out_dir, norm_field=True,
                     azimuth=-120, elevation=10)
#                      out_dir=out_dir, norm_field=True, 
#                      azimuth=240, elevation=5, img_resolution=600)
#         process_mesh(inp_path, suffix='_angl',
#                      out_dir=out_dir, norm_field=True, img_resolution=600)
#         process_mesh(inp_path, suffix='_pery',
#                      out_dir=out_dir, norm_field=True,
#                      azimuth=270, elevation=90,  img_resolution=600)
#         process_mesh(inp_path, suffix='_perz', 
#                      out_dir=out_dir, norm_field=True,
#                      azimuth=270, elevation=0, img_resolution=600)
#         process_mesh(inp_path, suffix='_perx', 
#                      out_dir=out_dir, norm_field=True,
#                      azimuth=180, elevation=0,  img_resolution=600)
        
#        process_mesh(inp_path, out_dir=out_dir, norm_field=True, model=model, colormap=colormap, img_resolution=600)
    else:
        print("No such file ", inp_path)

In [ ]:
inp_path = 'Expirements/OptimizationPaper/AfterMeeting/DeepSDFDrag/175/meshes/00039.ply'
data_instance = make_data_instance_from_ply(inp_path)
fields = model(data_instance.to('cuda:0')).cpu().detach().numpy()

process_mesh (inp_path , prefields=fields,
              out_dir="Expirements/Visualizations/Paper/OptimizationDifferent/175SpoilerDisappear", 
              norm_field=True, suffix='_spoiler', azimuth=-30, elevation=0, take_from_fld=False)

### Display Distributions

In [ ]:
mesh, fields = load_fld('/cvlabdata2/home/artem/Data/cars_refined/simulated/fld/0002_0005.fld')
print( np.min(fields[:, 0]), np.max(fields[:, 0]) )
norm_fields = (fields[:, 0] - np.mean(fields[:, 0])) / np.std(fields[:, 0])
print(np.min(norm_fields), np.max(norm_fields))
plt.hist(norm_fields, bins=100)
plt.show()

### Draw Colormap

In [ ]:
img = plt.imshow(np.array([[-6, 6]]), cmap="jet")
img.set_visible(False)

plt.colorbar(orientation="vertical")


In [ ]:
import pylab as pl
import numpy as np

a = np.array([[-1,1]])
pl.figure(figsize=(1, 9))
img = pl.imshow(a, cmap="jet")
pl.gca().set_visible(False)
cb = pl.colorbar(orientation="vertical", cax=pl.axes([0.1, 0.2, 0.4, 0.6]), ticks=[-0.8, 0, 0.8])
lines = cb.ax.tick_params(size = 0, width = 5)
pl.savefig("Expirements/Visualizations/Paper/PredictionComparison/jetColorMapOld/colorbar.png")

In [ ]:
len(lines[0].get_linewidths())

# Optimisation Progress

In [ ]:
root = '/cvlabdata2/home/artem/DeepSDF/Expirements/OptimizationPaper/CleanedDataBadDrag/'

for name in filter(lambda x: x[0] != '.' and x != 'DeepSDFDragFree', os.listdir(root)):
    result = 0
    num = 0
    exp_dir = os.path.join(root, name)
    for idx in filter(lambda x: x[0] != '.', os.listdir(exp_dir)):
        for step_id in [0, 10, 20, 29]:
            file_name = os.path.join(exp_dir, str(idx), 'meshes', str(step_id))
            print(file_name)